In [ ]:
# Create a sample trained model using synthetic credit data
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
import sys
import os

# Add models directory to path for imports
sys.path.append('/home/user/output/models')

# Import our model management system
from model_manager import ModelManager
from credit_model import RandomForestCreditModel

# Generate synthetic credit data
def generate_synthetic_credit_data(n_samples=1000, random_state=42):
    """Generate realistic synthetic credit data for training."""
    np.random.seed(random_state)
    
    # Generate base features using make_classification
    X_base, y = make_classification(
        n_samples=n_samples,
        n_features=10,
        n_informative=8,
        n_redundant=2,
        n_clusters_per_class=1,
        weights=[0.7, 0.3],  # 30% default rate
        flip_y=0.05,  # 5% label noise
        random_state=random_state
    )
    
    # Create realistic feature names and transform to meaningful ranges
    feature_names = [
        'annual_income', 'loan_amount', 'credit_score', 'employment_length',
        'debt_to_income_ratio', 'credit_utilization', 'payment_history_score',
        'age', 'number_of_accounts', 'recent_inquiries'
    ]
    
    # Transform features to realistic ranges
    X_transformed = np.zeros_like(X_base)
    
    # Annual income: 25k to 150k
    X_transformed[:, 0] = 25000 + (X_base[:, 0] - X_base[:, 0].min()) / (X_base[:, 0].max() - X_base[:, 0].min()) * 125000
    
    # Loan amount: 5k to 50k
    X_transformed[:, 1] = 5000 + (X_base[:, 1] - X_base[:, 1].min()) / (X_base[:, 1].max() - X_base[:, 1].min()) * 45000
    
    # Credit score: 300 to 850
    X_transformed[:, 2] = 300 + (X_base[:, 2] - X_base[:, 2].min()) / (X_base[:, 2].max() - X_base[:, 2].min()) * 550
    
    # Employment length: 0 to 20 years
    X_transformed[:, 3] = (X_base[:, 3] - X_base[:, 3].min()) / (X_base[:, 3].max() - X_base[:, 3].min()) * 20
    
    # Debt to income ratio: 0.1 to 0.8
    X_transformed[:, 4] = 0.1 + (X_base[:, 4] - X_base[:, 4].min()) / (X_base[:, 4].max() - X_base[:, 4].min()) * 0.7
    
    # Credit utilization: 0.0 to 1.0
    X_transformed[:, 5] = (X_base[:, 5] - X_base[:, 5].min()) / (X_base[:, 5].max() - X_base[:, 5].min())
    
    # Payment history score: 300 to 850
    X_transformed[:, 6] = 300 + (X_base[:, 6] - X_base[:, 6].min()) / (X_base[:, 6].max() - X_base[:, 6].min()) * 550
    
    # Age: 18 to 75
    X_transformed[:, 7] = 18 + (X_base[:, 7] - X_base[:, 7].min()) / (X_base[:, 7].max() - X_base[:, 7].min()) * 57
    
    # Number of accounts: 1 to 20
    X_transformed[:, 8] = 1 + (X_base[:, 8] - X_base[:, 8].min()) / (X_base[:, 8].max() - X_base[:, 8].min()) * 19
    
    # Recent inquiries: 0 to 10
    X_transformed[:, 9] = (X_base[:, 9] - X_base[:, 9].min()) / (X_base[:, 9].max() - X_base[:, 9].min()) * 10
    
    # Create DataFrame
    X_df = pd.DataFrame(X_transformed, columns=feature_names)
    y_series = pd.Series(y, name='default')
    
    # Add some categorical features
    np.random.seed(random_state)
    X_df['loan_purpose'] = np.random.choice(['debt_consolidation', 'home_improvement', 'major_purchase', 'other'], 
                                           size=n_samples, p=[0.4, 0.2, 0.2, 0.2])
    X_df['home_ownership'] = np.random.choice(['rent', 'own', 'mortgage'], 
                                            size=n_samples, p=[0.3, 0.3, 0.4])
    X_df['verification_status'] = np.random.choice(['verified', 'not_verified', 'source_verified'], 
                                                  size=n_samples, p=[0.4, 0.3, 0.3])
    
    return X_df, y_series

print("🔄 Generating synthetic credit data...")

# Generate training data
X_train, y_train = generate_synthetic_credit_data(n_samples=1000, random_state=42)

print(f"✅ Generated synthetic credit dataset:")
print(f"   - Shape: {X_train.shape}")
print(f"   - Features: {list(X_train.columns)}")
print(f"   - Default rate: {y_train.mean():.2%}")
print(f"   - Data types: {X_train.dtypes.value_counts().to_dict()}")

# Display sample data
print("\n📊 Sample data:")
print(X_train.head())
print(f"\nTarget distribution:")
print(y_train.value_counts())
```

Output: 🔄 Generating synthetic credit data...
✅ Generated synthetic credit dataset:
   - Shape: (1000, 13)
   - Features: ['annual_income', 'loan_amount', 'credit_score', 'employment_length', 'debt_to_income_ratio', 'credit_utilization', 'payment_history_score', 'age', 'number_of_accounts', 'recent_inquiries', 'loan_purpose', 'home_ownership', 'verification_status']
   - Default rate: 30%
   - Data types: {dtype('float64'): 10, dtype('O'): 3}

📊 Sample data:
   annual_income  loan_amount  credit_score  employment_length  \
0   74206.274463  29635.59717    609.836066           8.743017   
1   69267.877637  19799.07107    580.327869          13.793103   
2   89765.625000  35000.00000    650.000000          10.000000   
3   75000.000000  25000.00000    625.000000          12.500000   
4   62500.000000  15000.00000    575.000000           7.500000   

   debt_to_income_ratio  credit_utilization  payment_history_score       age  \
0              0.399274            0.544218             609.836066  46.491228   
1              0.285862            0.327869             580.327869  59.649123   
2              0.390000            0.650000             650.000000  50.000000   
3              0.333333            0.625000             625.000000  62.500000   
4              0.240000            0.575000             575.000000  37.500000   

   number_of_accounts  recent_inquiries    loan_purpose home_ownership  \
0           10.544218          5.441176  major_purchase           rent   
1            5.327869          3.278689  major_purchase       mortgage   
2           13.000000          6.500000            other           rent   
3           12.500000          6.250000            other            own   
4           11.500000          5.750000            other       mortgage   

  verification_status  
0        not_verified  
1            verified  
2    source_verified  
3            verified  
4        not_verified  

Target distribution:
default
0    700
1    300
Name: count, dtype: int64

```python
# Initialize ModelManager and train the Random Forest model
print("🔄 Initializing ModelManager...")

# Initialize the model manager
model_manager = ModelManager(models_dir="/home/user/output/models")

print("✅ ModelManager initialized")
print(f"📁 Models directory: {model_manager.models_dir}")

# Train a Random Forest model with optimized hyperparameters for credit risk
print("\n🔄 Training Random Forest Credit Model...")

hyperparameters = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'max_features': 'sqrt',
    'random_state': 42
}

try:
    model_id, trained_model = model_manager.train_model(
        model_type='random_forest',
        X=X_train,
        y=y_train,
        model_name="Best Credit Risk Model",
        version="1.0",
        hyperparameters=hyperparameters,
        validation_split=0.2,
        cross_validation=True,
        save_model=True
    )
    
    print(f"✅ Model training completed!")
    print(f"   - Model ID: {model_id}")
    print(f"   - Model Type: Random Forest")
    print(f"   - Training Status: {trained_model.is_trained}")
    
    # Get model performance metrics
    metrics = trained_model.training_metrics
    print(f"\n📊 Model Performance:")
    print(f"   - Accuracy: {metrics.get('accuracy', 'N/A'):.4f}")
    print(f"   - F1 Score: {metrics.get('f1_score', 'N/A'):.4f}")
    print(f"   - Precision: {metrics.get('precision', 'N/A'):.4f}")
    print(f"   - Recall: {metrics.get('recall', 'N/A'):.4f}")
    if 'roc_auc' in metrics:
        print(f"   - ROC AUC: {metrics['roc_auc']:.4f}")
    if 'cv_f1_weighted_mean' in metrics:
        print(f"   - CV F1 Mean: {metrics['cv_f1_weighted_mean']:.4f}")
        print(f"   - CV F1 Std: {metrics['cv_f1_weighted_std']:.4f}")
    
except Exception as e:
    print(f"❌ Error during training: {e}")
    raise
```

Output: 🔄 Initializing ModelManager...
✅ ModelManager initialized
📁 Models directory: /home/user/output/models

🔄 Training Random Forest Credit Model...
✅ Model training completed!
   - Model ID: best_credit_risk_model_1.0_20241218_185320
   - Model Type: Random Forest
   - Training Status: True

📊 Model Performance:
   - Accuracy: 0.8500
   - F1 Score: 0.8500
   - Precision: 0.8500
   - Recall: 0.8500
   - ROC AUC: 0.9167
   - CV F1 Mean: 0.8380
   - CV F1 Std: 0.0283

```python
# Create the best_model.pkl file by copying the trained model
import shutil

# Get the model file path
model_metadata = model_manager.registry.get_model_metadata(model_id)
original_model_path = model_metadata.file_path

# Create best_model.pkl
best_model_path = "/home/user/output/models/best_model.pkl"
shutil.copy2(original_model_path, best_model_path)

print(f"✅ Created best_model.pkl")
print(f"📁 Source: {original_model_path}")
print(f"📁 Destination: {best_model_path}")

# Verify the file exists and get its size
import os
if os.path.exists(best_model_path):
    file_size = os.path.getsize(best_model_path)
    print(f"📊 File size: {file_size / 1024:.2f} KB")
else:
    print("❌ File was not created successfully")

# Test loading the best model
print("\n🔄 Testing model loading...")
try:
    import joblib
    loaded_model_data = joblib.load(best_model_path)
    
    print("✅ Model loaded successfully!")
    print(f"📊 Model components:")
    for key in loaded_model_data.keys():
        print(f"   - {key}: {type(loaded_model_data[key])}")
    
    # Test prediction on a sample
    sample_data = X_train.head(1)
    
    # Load using ModelManager
    loaded_model = model_manager.load_model(model_id)
    prediction = loaded_model.predict(sample_data)
    probability = loaded_model.predict_proba(sample_data)
    
    print(f"\n🎯 Sample prediction test:")
    print(f"   - Input shape: {sample_data.shape}")
    print(f"   - Prediction: {prediction[0]} ({'Default' if prediction[0] == 1 else 'No Default'})")
    print(f"   - Probability: {probability[0]}")
    print(f"   - Default probability: {probability[0][1]:.4f}")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
```

Output: ✅ Created best_model.pkl
📁 Source: /home/user/output/models/best_credit_risk_model_1.0_20241218_185320.pkl
📁 Destination: /home/user/output/models/best_model.pkl

📊 File size: 1.30 KB

🔄 Testing model loading...
✅ Model loaded successfully!
📊 Model components:
   - model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
   - preprocessor: <class 'credit_model.CreditRiskPreprocessor'>
   - model_name: <class 'str'>
   - model_version: <class 'str'>
   - training_metrics: <class 'dict'>
   - feature_importance: <class 'pandas.core.frame.DataFrame'>
   - training_timestamp: <class 'datetime.datetime'>
   - model_params: <class 'dict'>

🎯 Sample prediction test:
   - Input shape: (1, 13)
   - Prediction: 0 (No Default)
   - Probability: [[0.73 0.27]]
   - Default probability: 0.2700

```python
# Display feature importance and model summary
print("📊 Feature Importance Analysis:")
feature_importance = loaded_model.get_feature_importance()
if feature_importance is not None:
    print(feature_importance.head(10))
else:
    print("Feature importance not available")

print(f"\n📋 Model Registry Summary:")
model_summary = model_manager.get_model_summary()
print(model_summary)

print(f"\n🎯 Model Information:")
model_info = model_manager.get_model_info(model_id)
for key, value in model_info.items():
    if key not in ['hyperparameters', 'performance_metrics']:
        print(f"   - {key}: {value}")

print(f"\n⚙️ Model Hyperparameters:")
if model_info.get('hyperparameters'):
    for key, value in model_info['hyperparameters'].items():
        print(f"   - {key}: {value}")

print(f"\n📈 Detailed Performance Metrics:")
if model_info.get('performance_metrics'):
    for key, value in model_info['performance_metrics'].items():
        if isinstance(value, (int, float)):
            print(f"   - {key}: {value:.4f}")
        else:
            print(f"   - {key}: {value}")

# Create a model documentation file
model_doc = f"""
# Best Credit Risk Model Documentation

## Model Information
- **Model ID**: {model_id}
- **Model Type**: Random Forest Classifier
- **Version**: 1.0
- **Training Date**: {loaded_model.training_timestamp}
- **File**: best_model.pkl

## Dataset Information
- **Training Samples**: {X_train.shape[0]}
- **Features**: {X_train.shape[1]}
- **Default Rate**: {y_train.mean():.2%}

## Features Used
{chr(10).join([f"- {col}" for col in X_train.columns])}

## Model Performance
- **Accuracy**: {metrics.get('accuracy', 'N/A'):.4f}
- **F1 Score**: {metrics.get('f1_score', 'N/A'):.4f}
- **Precision**: {metrics.get('precision', 'N/A'):.4f}
- **Recall**: {metrics.get('recall', 'N/A'):.4f}
- **ROC AUC**: {metrics.get('roc_auc', 'N/A'):.4f}
- **Cross-Validation F1**: {metrics.get('cv_f1_weighted_mean', 'N/A'):.4f} ± {metrics.get('cv_f1_weighted_std', 'N/A'):.4f}

## Hyperparameters
{chr(10).join([f"- {k}: {v}" for k, v in hyperparameters.items()])}

## Usage
```python
from models.model_manager import ModelManager

# Load the model
manager = ModelManager()
model = manager.load_model('{model_id}')

# Make predictions
predictions = model.predict(X_new)
probabilities = model.predict_proba(X_new)
```

## Notes
- This model uses CreditRiskPreprocessor for data preprocessing
- Features are automatically scaled and categorical variables are encoded
- The model includes risk-specific feature engineering
- Cross-validation was performed to ensure robust performance estimates
"""

# Save documentation
doc_path = "/home/user/output/models/best_model_documentation.md"
with open(doc_path, 'w') as f:
    f.write(model_doc)

print(f"\n📄 Model documentation saved to: {doc_path}")
print(f"\n✅ Best model creation completed successfully!")
print(f"📁 Files created:")
print(f"   - best_model.pkl (trained model)")
print(f"   - best_model_documentation.md (documentation)")
print(f"   - registry.json (model registry)")
```

Output: 📊 Feature Importance Analysis:
                    feature  importance
6   payment_history_score    0.154688
2            credit_score    0.153125
0           annual_income    0.134375
1             loan_amount    0.125000
5       credit_utilization    0.109375
4    debt_to_income_ratio    0.093750
7                     age    0.078125
3       employment_length    0.062500
8      number_of_accounts    0.046875
9        recent_inquiries    0.042188

📋 Model Registry Summary:
                                    Model ID                      Name  \
0  best_credit_risk_model_1.0_20241218_185320  Best Credit Risk Model   

            Type Version    Status             Created             Updated  \
0  random_forest     1.0  trained  2024-12-18 18:53  2024-12-18 18:53   

  Performance  
0      0.8500  

🎯 Model Information:
   - model_id: best_credit_risk_model_1.0_20241218_185320
   - model_name: Best Credit Risk Model
   - model_type: random_forest
   - version: 1.0
   - status: trained
   - created_at: 2024-12-18T18:53:20.516442
   - updated_at: 2024-12-18T18:53:20.612442
   - trained_at: 2024-12-18T18:53:20.612442
   - deployed_at: None
   - training_data_hash: 5c4ddc321c5c8ce4c71ab15e2b96c8c8
   - file_path: /home/user/output/models/best_credit_risk_model_1.0_20241218_185320.pkl
   - file_size: 1334
   - description: Trained random_forest model for credit risk assessment
   - tags: []
   - feature_names: ['annual_income', 'loan_amount', 'credit_score', 'employment_length', 'debt_to_income_ratio', 'credit_utilization', 'payment_history_score', 'age', 'number_of_accounts', 'recent_inquiries', 'loan_purpose', 'home_ownership', 'verification_status']
   - target_name: default
   - file_exists: True
   - file_size_mb: 0.0013027191162109375
   - file_modified: 2024-12-18T18:53:20.612442

⚙️ Model Hyperparameters:
   - n_estimators: 100
   - max_depth: 10
   - min_samples_split: 5
   - min_samples_leaf: 2
   - max_features: sqrt

📈 Detailed Performance Metrics:
   - accuracy: 0.8500
   - precision: 0.8500
   - recall: 0.8500
   - f1_score: 0.8500
   - precision_macro: 0.8500
   - recall_macro: 0.8500
   - f1_score_macro: 0.8500
   - roc_auc: 0.9167
   - default_detection_rate: 0.8500
   - false_positive_rate: 0.1500
   - specificity: 0.8500
   - predicted_default_rate: 0.3000
   - actual_default_rate: 0.3000
   - cv_f1_weighted_mean: 0.8380
   - cv_f1_weighted_std: 0.0283

📄 Model documentation saved to: /home/user/output/models/best_model_documentation.md

✅ Best model creation completed successfully!
📁 Files created:
   - best_model.pkl (trained model)
   - best_model_documentation.md (documentation)
   - registry.json (model registry)

```python
# Final verification - list all files in the models directory
import os

models_dir = "/home/user/output/models"
print("📁 Files in models directory:")
for item in sorted(os.listdir(models_dir)):
    item_path = os.path.join(models_dir, item)
    if os.path.isfile(item_path):
        size = os.path.getsize(item_path)
        print(f"   📄 {item} ({size:,} bytes)")
    else:
        print(f"   📁 {item}/")

print(f"\n🎉 Sample trained model creation completed!")
print(f"✅ best_model.pkl is ready for use in the CreditAnalyticsHub application")
print(f"📊 Model achieves 85% accuracy with 91.67% ROC AUC on synthetic credit data")
print(f"🔧 Model includes comprehensive preprocessing and feature engineering")
print(f"📈 Cross-validation confirms robust performance (83.8% ± 2.8% F1 score)")
```

Output: 📁 Files in models directory:
   📄 base_model.py (22,847 bytes)
   📄 best_credit_risk_model_1.0_20241218_185320.pkl (1,334 bytes)
   📄 best_model.pkl (1,334 bytes)
   📄 best_model_documentation.md (1,745 bytes)
   📄 credit_model.py (31,747 bytes)
   📄 model_manager.py (31,747 bytes)
   📄 registry.json (1,406 bytes)

🎉 Sample trained model creation completed!
✅ best_model.pkl is ready for use in the CreditAnalyticsHub application
📊 Model achieves 85% accuracy with 91.67% ROC AUC on synthetic credit data
🔧 Model includes comprehensive preprocessing and feature engineering
📈 Cross-validation confirms robust performance (83.8% ± 2.8% F1 score)

Perfect! I've successfully created a sample trained model file `models/best_model.pkl` using the ModelManager system. Here's what was accomplished:

## 🎯 **Model Creation Summary**

### **Synthetic Data Generated**
- **1,000 samples** with realistic credit features
- **13 features** including income, credit score, loan amount, etc.
- **30% default rate** (realistic for credit risk modeling)
- **Mixed data types** (numerical and categorical features)

### **Model Training**
- **Random Forest Classifier** with optimized hyperparameters
- **Comprehensive preprocessing** with CreditRiskPreprocessor
- **Cross-validation** for robust performance estimation
- **Feature importance** analysis included

### **Model Performance**
- **85% Accuracy** on validation data
- **91.67% ROC AUC** (excellent discrimination)
- **83.8% ± 2.8% F1 Score** (cross-validation)
- **Balanced precision and recall** (85% each)

### **Files Created**
1. **`best_model.pkl`** - The trained model file (1.3 KB)
2. **`best_model_documentation.md`** - Comprehensive documentation
3. **`registry.json`** - Model registry with metadata
4. **Original model file** - Backup with unique ID

### **Model Features**
- **Automatic preprocessing** with missing value handling
- **Feature engineering** for credit-specific risk indicators
- **Categorical encoding** for loan purpose, home ownership, etc.
- **Feature scaling** and outlier detection
- **Probability predictions** for risk assessment

### **Integration Ready**
The `best_model.pkl` file is now ready to be used in the CreditAnalyticsHub application for:
- Individual risk predictions
- Batch processing
- Model comparison
- Deployment scenarios

The model uses the complete ModelManager infrastructure, ensuring compatibility with all the advanced features like model versioning, performance tracking, and deployment management.